In [1]:
import os

In [2]:
%pwd

'd:\\End_to_End\\textsummarisation\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\End_to_End\\textsummarisation'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_PATH):

        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name
        )
        
        return model_evaluation_config
    



In [12]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import evaluate
import torch
import pandas as pd
from tqdm import tqdm

In [15]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config
    
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split dataset into smaller batches that we can process simultaneously
        yield successive batch_size chunks from list_of_elements"""

        for i in range(0,len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]
        
    
    def calculate_metric_on_test_ds(self,dataset,metric,model,tokenizer,
                                batch_size=16,device="cuda" if torch.cuda.is_available() else "cpu",
                                column_text="article",
                                column_summary="highlights"):
        article_batches=list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches=list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch,target_batch in tqdm(
            zip(article_batches,target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=512, truncation=True, padding="max_length", return_tensors="pt")

            summaries= model.generate(input_ids=inputs['input_ids'].to(device),
                            attention_mask=inputs['attention_mask'].to(device),
                            length_penalty=0.8, num_beams=8, max_length=128)
            
            '''parameter for lengthy penalty ensure that the model doesn't generate too lengthy sentences'''

            #Finally we doceded the generated texts
            #replace the token, and add the decoded texts with reference to the metrics

            decode_summaries = [tokenizer.decode(s, skip_special_tokens=True,clean_up_tokenization_spaces=True) for s in summaries]

            decode_summaries=[d.replace("", " ") for d in decode_summaries]

            metric.add_batch(predictions=decode_summaries, references=target_batch)
        
        #finally compute and return the ROUGE scores

        score = metric.compute()

        return score
    
    def evaluate(self):

        device="cuda" if torch.cuda.is_available() else "cpu"
        tokenizer=AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_t5=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path)

        #loading data

        dataset_samsusm_pt=load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

        rouge_metric=evaluate.load('rouge')

        score=self.calculate_metric_on_test_ds(
            dataset_samsusm_pt['test'][0:10], rouge_metric,model_t5, tokenizer,batch_size=2, column_text= 'dialogue', column_summary= 'summary'
        )

        rouge_dict={rn:score[rn] for rn in rouge_names}

        df=pd.DataFrame(rouge_dict, index=['t5-model'])

        df.to_csv(self.config.metric_file_name, index=False)
        
    

In [16]:
try:
    config = ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation=ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise e

[2025-11-14 15:27:41,164: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-14 15:27:41,169: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-14 15:27:41,172: INFO: common: created directory at: artifacts]
[2025-11-14 15:27:41,175: INFO: common: created directory at: artifacts/model_evaluation]


100%|██████████| 5/5 [01:24<00:00, 16.92s/it]

[2025-11-14 15:29:10,087: INFO: rouge_scorer: Using default tokenizer.]
